<center>

<h1 style="text-align:center"> Modular Programming </h1>
<h2 style="text-align:center"> CS3100 Monsoon 2020 </h2>
</center>

## Review

* Previously
  + How to build small programs
  
* This lecture
  + How to build at scale: Structures, Signatures, Functors.
  
$
\require{color}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
\newcommand{\cgreen}[1]{{\color{\green}{\text{#1}}}}
$

## Scale

* **Assignment 1 & 2** : ~100 lines of code
* **OCaml** : 375000 lines of code
* **Hubble space telescope** : 2 million lines of code
* **Facebook** : 60 million lines of code (estimated)
* **Google (all services)** : 2 billion lines of code (estimated)

https://informationisbeautiful.net/visualizations/million-lines-of-code/

...can’t be done by one person

...no individual programmer can understand all the details

...$\cred{too complex to build with OCaml we’ve seen so far}$

## Modularity

* Code comprises of individual **Modules**
* Developed separately
  + Reason $\cgreen{locally}$, not $\cred{globally}$.
* Clearly specified **Interfaces** for using the modules

<center>

<img src="modules.png" width=250>
</center>

## Features of modularity

* **Namespacing**
  + Provide way to name a collection of related features (`List`, `Set`)
  + Avoid name clashes with similarly named features from different collections (`insert` in `List` and `Set`)

## Features of modularity

* **Abstraction**
  + Hide the details of implementation
  + Avoid the user breaking invariants implicit in the code.
    + Consider using a `List` as a stack (FIFO)
    + User sorts the list which has no meaning in stacks.
  + Transparently change the implementation without breaking the code of the client (i.e. user) of the module.
    + Implement stack with a `List` rather than an `Array`

## Features of modularity

* **Code reuse**
  + Avoid reimplementing features that are already present.
  + Using a list as a stack (FIFO): Reuse functions such as `is_empty` and `length`.

## OCaml Features for Modularity

* **Namespacing:** Structure
* **Abstraction:** Signature
* **Code Reuse:** Functors, includes, and sharing constraints.

## Structure

* A collection of **definitions**.
* Evaluated in order
* Structure value can be bound to a name

The syntax is 

```ocaml
module Module_name = struct 
  (* collection of definitions *) 
end
```

## Structure

+ Let us implement a purely functional stack using OCaml's built-in list data structure.
+ In a purely functional data structure, "update" operations return a new version of the data structure.
  + In a stack, push and pop return a new version of the stack.

In [ ]:
module Stack = struct
  let empty = []
  let push v s = v::s
  let pop s = match s with
  | [] -> None
  | x::xs -> Some (x, xs)
  let depth s = List.length s
end

## Structures provide namespacing

In [ ]:
Stack.empty

In [ ]:
empty

## Signature

* A collection of **declarations**.
* No evaluation, only used for type-checking. 
* Signature type can also be bound to a name. 

```ocaml
module type Module_type_name = sig
  (* collection of declaration *)
end
```

## Stack Signature

In [ ]:
module type StackType = sig
  val empty : 'a list
  val push : 'a -> 'a list -> 'a list
  val pop : 'a list -> ('a * 'a list) option
  val depth : 'a list -> int
end

## Handy Jupyter Functions 

Jupyter (really the OCaml top-level power Jupyter) has the following handy functions to display the modules and module signatures.

In [ ]:
#show_module Stack

In [ ]:
#show_module_type StackType

## Explicit Signatures

In [ ]:
module M : StackType = Stack

## Hiding functionality with explicit signatures

In [ ]:
module M : (sig 
  val empty : 'a list 
end) = Stack

In [ ]:
Stack.empty;;
Stack.depth

In [ ]:
M.empty;;
M.depth

## Opening modules

* Use `open` to make available all the definitions from the module

In [ ]:
open Stack

In [ ]:
empty

## Opening shadows earlier defintions

In [ ]:
module M = struct let x = 10 end
module N = struct let x = 20 end
open M
open N

In [ ]:
x

## Including module functionality

* `include` allows new modules to be constructed by extending earlier modules (Code reuse).

In [ ]:
module Stack = struct
  include Stack
  let is_empty s = match s with
  | [] -> true
  | _ -> false
end

## Include on signatures

`include` also works on signatures to define other signatures.

In [ ]:
module type MT = sig
  include StackType
  val is_empty : 'a list -> bool
end

## Difference between Open and Include

In [ ]:
module M1 = struct open Stack end;;
module M2 = struct include Stack end

```ocaml
module M1 = struct open Stack end
``` 
does not export the fields of the `Stack` module, whereas
```ocaml
module M1 = struct include Stack end
``` 
does.

## Abstract types

* So far we have only seen how to collect definitions under a common name (Namespacing)
  + The implementation details are still visible.

In [ ]:
let s = Stack.empty |> Stack.push 1 |> Stack.push 2 |> Stack.push 3

`s` is of type `int list`

Can do non-sensical operations for a stack such as `List.sort`.

In [ ]:
List.sort (-) s

## Abstract stack

**Abstract** the type of stack such that only those operations allowed by the interface are applicable. 

Define the stack signature with abstract stack type.

In [4]:
module type AbsStackType = sig
  type 'a t (* abstract type *)
  val empty    : 'a t
  val push     : 'a -> 'a t -> 'a t
  val pop      : 'a t -> ('a * 'a t) option
  val depth    : 'a t -> int
  val is_empty : 'a t -> bool
end

module type AbsStackType =
  sig
    type 'a t
    val empty : 'a t
    val push : 'a -> 'a t -> 'a t
    val pop : 'a t -> ('a * 'a t) option
    val depth : 'a t -> int
    val is_empty : 'a t -> bool
  end


## Abstract stack

In [ ]:
module AbsStack : AbsStackType = struct
  type 'a t = 'a list
  include Stack
end

In [ ]:
let s = AbsStack.empty |> AbsStack.push 1 |> AbsStack.push 2 |> AbsStack.push 3

In [ ]:
AbsStack.pop s

## Abstraction

* Interfaces with abstract types are **contracts** between the *implementer* and the *user*.
* User cannot violate the API
  + No way to sort the internal list anymore.
* Implementer can transparently change the internals of the implementation 
  + Implement the stack with an array rather than list
  + As long as the same API is preserved, the user code does not break.

## Stack using user-defined list

In [3]:
module VariantStack = struct
  type 'a t = 
  | Nil 
  | Cons of 'a * 'a t
  
  let empty = Nil
  
  let rec depth_aux acc l = match l with
  | Nil -> acc
  | Cons (x, xs) -> depth_aux (1+acc) xs
  
  let depth l = depth_aux 0 l
  
  let push v s = Cons (v,s)
  
  let pop s = match s with Nil -> None | Cons (x,xs) -> Some (x,xs)
  
  let is_empty s = match s with Nil -> true | _ -> false
end

module VariantStack :
  sig
    type 'a t = Nil | Cons of 'a * 'a t
    val empty : 'a t
    val depth_aux : int -> 'a t -> int
    val depth : 'a t -> int
    val push : 'a -> 'a t -> 'a t
    val pop : 'a t -> ('a * 'a t) option
    val is_empty : 'a t -> bool
  end


## Abstracting the variant stack

In [5]:
module AbsVariantStack : AbsStackType = VariantStack

module AbsVariantStack : AbsStackType


Wherever the user uses the `AbsStack`, we can replace that with `AbsVariantStack` and the user code wouldn't be able to tell the difference. 

## Two languages

* OCaml is a stratified language.
  + Values + Expressions and Types at term level.
  + Structures and Signatures at module level.
* Structures are (generally) not first-class in OCaml.
  + OCaml has **first-class modules**, which we will not cover in this class.
* What is the equivalent of functions at the module level?
  + And why would we need it?
* **Functors**
  + Functions that take structures and return other structures.

## Identity functor

The simplest function is the identity function

In [10]:
let id (x : int) : int = x

val id : int -> int = <fun>


At the module level, we can correspondingly define

In [12]:
module type T = sig 
  type t 
  val v : t 
end

module type T = sig type t val v : t end


In [13]:
module Id (X : T) : T = X

module Id : functor (X : T) -> T


## Applying Identity functor

In [14]:
id 5

- : int = 5


Similarly:

In [15]:
module M = struct 
  type t = int
  let v = 10
end

module M : sig type t = int val v : int end


In [16]:
module M' = Id(M)

module M' : sig type t = Id(M).t val v : t end


## Type equality under abstraction

* We know `M` and `M'` are the same modules.
  + Let's ask whether `M.v = M'.v`.

In [17]:
M.v = M'.v

error: compile_error

* While the compiler knows that the type `M.t` is `int`, the type `M'.t` (return type of functor) is **abstract**
  + Compiler does not know that `M.t` is the same type as `M'.t`. 

## Sharing constraints

* Use **sharing constraints** to let the compiler know about type equalities.
* Sharing constraints make the module types _less abstract / more concrete_. 

Consider the original identity functor definition

In [18]:
module Id (X : T) : T = X

module Id : functor (X : T) -> T


Now consider the identity functor with sharing constraint on the return type `T`

In [20]:
module Id2 (X : T) : (T with type t = X.t)
  = X

module Id2 : functor (X : T) -> sig type t = X.t val v : t end


## Sharing Constraints

In [23]:
module M2 = Id2(M)

module M2 : sig type t = M.t val v : t end


In [24]:
M2.v = M.v

- : bool = true


## Multiple sharing constraints

In [25]:
module type T2 = sig 
  type t
  type u 
end

module Id2 (X2 : T2) : 
  (T2 with type t = X2.t
       and type u = X2.u) = struct 
  type t = X2.t
  type u = X2.u
end

module type T2 = sig type t type u end


module Id2 : functor (X2 : T2) -> sig type t = X2.t type u = X2.u end


## Functor Example: Serializable List

* Write a `string_of_list` and `list_of_string` functions for _any typed_ list.
    + We need to know how to convert the list element to string and vice-versa.
* For an `int list`, we can write:

In [26]:
let string_of_list l = 
  let rec loop acc l = match l with
    | [] -> acc
    | [x] -> acc ^ (string_of_int x)
    | x::xs -> loop (acc ^ (string_of_int x) ^ ";") xs
  in
  "[" ^ (loop "" l) ^ "]"  

val string_of_list : int list -> string = <fun>


In [27]:
string_of_list [1;2;3]

- : string = "[1;2;3]"


## Functor Example: Serializable List

In [28]:
let list_of_string s =
  let s = String.sub s 1 (String.length s - 2) in
  List.map int_of_string (String.split_on_char ';' s)

val list_of_string : string -> int list = <fun>


In [29]:
list_of_string "[1;2;3]"

- : int list = [1; 2; 3]


## Generalise

What about float, tuples, records, etc? Use **Functors**.

In [30]:
module type Serializable = sig
  type t
  val t_of_string : string -> t
  val string_of_t : t -> string
end

module type Serializable =
  sig type t val t_of_string : string -> t val string_of_t : t -> string end


## Generalise

What about float, tuples, records, etc? Use **Functors**.

In [31]:
module SerializableList (C : Serializable) = struct
  type t = C.t list
  
  let string_of_t l = 
    let rec loop acc l = match l with
      | [] -> acc
      | [x] -> acc ^ (C.string_of_t x)
      | x::xs -> loop (acc ^ (C.string_of_t x) ^ ";") xs
    in
    "[" ^ (loop "" l) ^ "]"  
    
  let t_of_string s =
    let s = String.sub s 1 (String.length s - 2) in
    List.map C.t_of_string (String.split_on_char ';' s)
end

module SerializableList :
  functor (C : Serializable) ->
    sig
      type t = C.t list
      val string_of_t : C.t list -> string
      val t_of_string : string -> C.t list
    end


## SerializableFloatList

In [32]:
module SerializableFloatList = SerializableList (struct 
  type t = float
  let t_of_string = float_of_string
  let string_of_t = string_of_float
end)

module SerializableFloatList :
  sig
    type t = float list
    val string_of_t : float list -> string
    val t_of_string : string -> float list
  end


## SerializableFloatList

In [33]:
SerializableFloatList.string_of_t [1.4;2.3;3.4]

- : string = "[1.4;2.3;3.4]"


In [34]:
SerializableFloatList.t_of_string "[1.4;2.3;3.4]"

- : float list = [1.4; 2.3; 3.4]


## SerializableStringList

In [35]:
module SerializableStringList = SerializableList (struct 
  type t = string
  let t_of_string x = x
  let string_of_t x = x
end)

module SerializableStringList :
  sig
    type t = string list
    val string_of_t : string list -> string
    val t_of_string : string -> string list
  end


## SerializableStringList

In [36]:
SerializableStringList.string_of_t ["Et";"tu,";"Brute?"]

- : string = "[Et;tu,;Brute?]"


In [37]:
SerializableStringList.t_of_string "[Et;tu,;Brute?]"

- : string list = ["Et"; "tu,"; "Brute?"]


## Behold the power of abstraction

Observe that the signature of `SerializableFloatList` is also `Serializable`.

In [38]:
module SerializableFloatListList = SerializableList (SerializableFloatList)

module SerializableFloatListList :
  sig
    type t = SerializableFloatList.t list
    val string_of_t : SerializableFloatList.t list -> string
    val t_of_string : string -> SerializableFloatList.t list
  end


In [39]:
SerializableFloatListList.string_of_t [[1.1]; [2.1;2.2]; [3.1;3.2;3.3]]

- : string = "[[1.1];[2.1;2.2];[3.1;3.2;3.3]]"


<center>

<h1 style="text-align:center"> Fin. </h1>
</center>